In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import gc
from sklearn.metrics import f1_score, accuracy_score, mean_absolute_error
import lightgbm as lgb
import random
import pickle
from tqdm import tqdm_notebook
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
def BT_filter(values):
    BT_mean = []
#     values = df.values
    for idx in range(len(values)):
        val_0 = idx-41
        val_1 = idx-40
        val_2 = idx-39
        val_3 = idx-1
        val_4 = idx
        val_5 = idx+1
        val_6 = idx+39
        val_7 = idx+40
        val_8 = idx+41
        
        val_4 = values[idx]
        if val_0<0 or (val_0%40)==39:
            val_0 = -1
        else:
            val_0 = values[val_0]
            
        if val_1<0:
            val_1 = -1
        else:
            val_1 = values[val_1]
            
        if val_2<0 or (val_2%40) ==0:
            val_2 = -1
        else:
            val_2 = values[val_2]
            
        if (val_3%40)==39:
            val_3 = -1
        else:
            val_3 = values[val_3]
            
        if (val_5%40)==0:
            val_5 = -1
        else:
            val_5 = values[val_5]
            
        if (val_6%40)==39 or val_6>=1600:
            val_6 = -1
        else:
            val_6 = values[val_6]
            
        if val_7>=1600:
            val_7 = -1
        else:
            val_7 = values[val_7]
            
        if (val_8%40)==0 or val_8>=1600:
            val_8 = -1
        else:
            val_8 = values[val_8]

        vals = np.array([val_0, val_1, val_2, val_3, val_4, val_5, val_6, val_7, val_8])
        vals = np.array([k for k in vals if k>=0])
        BT_mean.append(vals.mean())
    return BT_mean

In [ ]:
# %%time
# train_files = glob.glob('dataset/train/*.npy')          # train 파일들 로드
test_files = glob.glob('dataset/test/*.npy')            # test 파일들 로드

# train_files = sorted(train_files)                          # 파일명으로 정렬
test_files = sorted(test_files)                          # 파일명으로 정렬

# submission = pd.read_csv('dataset/sample_submission.csv')

# train = pd.read_feather('dataset/train_missing_32.feather')              # missing_value가 있는 값들 없애고 32
# test = pd.read_feather('dataset/test_missing_32.feather')              # missing_value가 있는 값들 없애고 32

# train = train.iloc[int(train.shape[0]*0.5):]

In [ ]:
# dfs = []
# num=1
# for idx in tqdm_notebook(range(len(train_files[7634*(num-1):7634*num]))):
#     df = pd.DataFrame(np.load(train_files[idx]).reshape(1600,-1))
#     df.columns = ['BT1', 'BT2', 'BT3' ,'BT4' ,'BT5', 'BT6', 'BT7', 'BT8', 'BT9', 'type', 'GMI_lon', 'GMI_lat', 'DPR_lon', 'DPR_lat', 'target']
#     for bt in ['BT1', 'BT2', 'BT3' ,'BT4' ,'BT5', 'BT6', 'BT7', 'BT8', 'BT9']:
#         df[bt+'_mean'] = BT_filter(df[bt].values)
#     dfs.append(df)

In [ ]:
# BT_mean_column = ['BT1','BT2','BT3','BT4','BT5','BT6','BT7','BT8','BT9']
# for col1 in range(len(BT_mean_column)):
#     for col2 in (range(col1, len(BT_mean_column), 1)):
#         if col1!=col2:
#             train[str(BT_mean_column[col1])+'-'+str(BT_mean_column[col2])] = train[BT_mean_column[col1]] - train[BT_mean_column[col2]]

In [ ]:
BT_mean_column = ['BT1_mean','BT2_mean','BT3_mean','BT4_mean','BT5_mean','BT6_mean','BT7_mean','BT8_mean','BT9_mean']
for col1 in range(len(BT_mean_column)):
    for col2 in (range(col1, len(BT_mean_column), 1)):
        if col1!=col2:
            train[str(BT_mean_column[col1])+'-'+str(BT_mean_column[col2])+'_mean'] = train[BT_mean_column[col1]] - train[BT_mean_column[col2]]

In [ ]:
# BT_mean_column = ['BT1_mean','BT2_mean','BT3_mean','BT4_mean','BT5_mean','BT6_mean','BT7_mean','BT8_mean','BT9_mean']
# for col1 in range(len(BT_mean_column)):
#     for col2 in (range(col1, len(BT_mean_column), 1)):
#         if col1!=col2:
#             test[str(BT_mean_column[col1])+'-'+str(BT_mean_column[col2])] = test[BT_mean_column[col1]] - test[BT_mean_column[col2]]

In [ ]:
# dfs = []
# for idx in tqdm_notebook(range(len(test_files))):
#     df = pd.DataFrame(np.load(test_files[idx]).reshape(1600,-1))
#     df.columns = ['BT1', 'BT2', 'BT3' ,'BT4' ,'BT5', 'BT6', 'BT7', 'BT8', 'BT9', 'type', 'GMI_lon', 'GMI_lat', 'DPR_lon', 'DPR_lat']
#     for bt in ['BT1', 'BT2', 'BT3' ,'BT4' ,'BT5', 'BT6', 'BT7', 'BT8', 'BT9']:
#         df[bt+'_mean'] = BT_filter(df[bt].values)
#     dfs.append(df)


# dfs = []
# for idx in tqdm_notebook(range(len(test_files))):
#     df = pd.DataFrame(np.load(test_files[idx]).reshape(1600,-1))
#     df.columns = ['BT1', 'BT2', 'BT3' ,'BT4' ,'BT5', 'BT6', 'BT7', 'BT8', 'BT9', 'type', 'GMI_lon', 'GMI_lat', 'DPR_lon', 'DPR_lat']
#     dfs.append(df)


In [ ]:
# 데이터 저장
test = pd.concat(dfs)
test = test.astype(np.float32).reset_index(drop=True)
# test = test.drop(['BT1', 'BT2', 'BT3' ,'BT4' ,'BT5', 'BT6', 'BT7', 'BT8', 'BT9', 'type', 'GMI_lon', 'GMI_lat', 'DPR_lon', 'DPR_lat'],axis=1)
test.reset_index(drop=True, inplace=True)
test.to_feather('dataset/test_missing_btmean.feather')

In [ ]:
%%time
with open('dataset/y_train_BT+mean_processing.pickle', 'wb') as f:
    pickle.dump(y_train, f)
with open('dataset/y_valid_BT+mean_processing.pickle', 'wb') as f:
    pickle.dump(y_valid, f)
X_train.reset_index(drop=True).to_feather('dataset/X_train_BT+mean_processing.feather')
X_valid.reset_index(drop=True).to_feather('dataset/X_valid_BT+mean_processing.feather')

In [ ]:
%%time
# with open('dataset/y_train_BT+mean_processing.pickle', 'rb') as f:
#     y_train = pickle.load(f)
# with open('dataset/y_valid_BT+mean_processing.pickle', 'rb') as f:
#     y_valid = pickle.load(f)
# print('loaded')
# X_train_1of2 = pd.read_feather('dataset/X_train_BT+mean_processing_1of2.feather')
# print('loaded')
# X_train_2of2 = pd.read_feather('dataset/X_train_BT+mean_processing_2of2.feather')
# print('loaded')
# X_valid_1of2 = pd.read_feather('dataset/X_valid_BT+mean_processing_1of2.feather')
# print('loaded')
# X_valid_2of2 = pd.read_feather('dataset/X_valid_BT+mean_processing_2of2.feather')